In [6]:
import os
import sys
import pandas as pd
import numpy as np
from rank_bm25 import BM25Okapi

caminho_commons = os.path.join('..', '..', 'ajna_docs', 'commons')
caminho_virasana = os.path.join('..', '..', 'ajna_docs', 'virasana')
sys.path.append(caminho_commons)
sys.path.append('..')
sys.path.append(caminho_virasana)
from bhadrasana.models import engine

In [7]:
SQL = 'SELECT * FROM ovr_itenstg'
df = pd.read_sql(SQL, engine)

In [8]:
df

,id,tg_id,numero,descricao,qtde,unidadedemedida,valor,ncm,marca_id,create_date,user_name,contramarca,modelo
0,1,2,1,Rolamento automativo power guido,189024.0,0,200.00,,7.0,2020-03-06 12:16:32,None,None,None
1,2,9,1,123,1.0,1,1.00,11,NaN,2020-05-26 18:44:26,None,None,None
2,9,54,1,MÁSCARA FACIAL HIDRATANTE EM EMBALAGEM DE 25ML...,226000.0,0,2.60,33049910,NaN,2020-08-04 10:44:00,None,None,None
3,112,123,0,DIOCTIL FTALATO EM FLEXI-BAG D/C 20 TONELADAS ...,2.0,0,18000.00,29173200,NaN,2020-08-31 08:25:48,None,None,None
4,121,125,0,"SOLA DE CALÇADO, DE BORRACHA, N 39 A, REF XL B...",462.0,0,21.21,64062000,NaN,2020-09-11 14:32:05,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13888,18692,626,1,Pólvora - Hodgon CFE 223 - 3630g,2.0,0,0.00,None,NaN,2022-04-26 14:29:44,None,None,None
13889,18693,626,2,Projétil - Nosler - Cal30. 190 g.,250.0,0,0.00,None,NaN,2022-04-26 14:29:44,None,None,None
13890,18694,626,3,Pólvora - Accurate 2015 - 454g,2.0,0,0.00,None,NaN,2022-04-26 14:29:44,None,None,None
13891,18695,626,4,Projétil - Berger - Cal30. 185 g.,200.0,0,0.00,None,NaN,2022-04-26 14:29:44,None,None,None


In [9]:
itenstg = df.dropna(subset=['ncm'])

In [10]:
len(itenstg)

13547

In [20]:
import nltk
import unicodedata

def remove_accents(input_str):  
    """ 
    Função converte string em bytes, mas antes normaliza string usando NFKD
    
    NFKD - decompõem em dois code point e analisa compatibilidades (sem ser canonicamente equivalente)
    https://docs.python.org/3/library/unicodedata.html
    """
    nfkd_form = unicodedata.normalize('NFKD', input_str.lower())
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii


def remove_pontuacao(doc):
    return ''.join([leter for leter in doc if leter not in ['.', ',', ':', ';', '-', '+']])

stop_words = nltk.corpus.stopwords.words('portuguese')

def tokenize(doc):
    doc_normalized = doc.lower()
    doc_normalized = remove_pontuacao(doc_normalized)
    doc_normalized = remove_accents(doc_normalized)
    return [word for word in doc_normalized.split() if (len(word) > 2 and word not in stop_words)]

In [12]:
tokenize('KIT DE CARTEIRA, COM RELÓGIO DE PULSO; Modelo: bolinha, Marca:bolão')

[b'kit',
 b'carteira',
 b'com',
 b'relogio',
 b'pulso',
 b'modelo',
 b'bolinha',
 b'marcabolao']

In [13]:
corpus = list(itenstg.descricao.values)
corpus_normalized = [tokenize(line) for line in corpus]
bm25n = BM25Okapi(corpus_normalized)
bm25n.get_top_n(tokenize('KIT DE CARTEIRA, COM RELÓGIO DE PULSO;'), corpus, n=10)

['KIT DE CARTEIRA COM RELÓGIO DE PULSO DE PONTEIRO, IMITAÇÃO DOS PERSONAGENS CARS, DA MARCA DISNEY – REF.:M301A',
 'KIT DE CARTEIRA COM RELÓGIO DE PULSO DE PONTEIRO, IMITAÇÃO DA PERSONAGEM FROZEN, DA MARCA DISNEY – REF.: M301A',
 'Kit de carteira com relógio imitação do personagem Miraculous . (Ref:M301A)',
 'KIT DE CARTEIRA COM RELÓGIO DE PULSO DE PONTEIRO, IMITAÇÃO DA PERSONAGEM MIRACULOUS, DA MARCA GLOBOSAT PROGRAMADORA – REF.: M301A',
 'KIT DE CARTEIRA COM RELÓGIO DE PULSO DE PONTEIRO, IMITAÇÃO DA PERSONAGEM PRINCESA SOFIA, DA MARCA DISNEY – REF.: M301A',
 'KIT DE CARTEIRA COM RELÓGIO DE PULSO DE PONTEIRO, IMITAÇÃO DO PERSONAGEM HOMEM ARANHA, DA MARCA MARVEL – REF.: M301A',
 'KIT DE CARTEIRA COM RELÓGIO DE PULSO DE PONTEIRO, IMITAÇÃO DOS PERSONAGENS AVENGERS, DA MARCA MARVEL – REF.: M301A',
 'KIT DE CARTEIRA COM RELÓGIO DE PULSO DE PONTEIRO, IMITAÇÃO DO PERSONAGEM BEN10, DA MARCA CARTOON NETWORK – REF.: M301A',
 'KIT DE CARTEIRA COM RELÓGIO DE PULSO DE PONTEIRO, IMITAÇÃO DAS PERSON

In [14]:
bm25n.get_top_n(tokenize('CONFECÇÕES USO ADULTO, LACOSTE, THOMMY HILFGER RALPH LAUREN E OUTRAS DIVERSAS'), corpus, n=10)

['CONFECÇÕES ADULTO LACOSTE, TOMMY HILFGER RALPH LAUREN E OUTRAS MARCAS',
 'CONFECÇÕES USO ADULTO, LACOSTE, THOMMY HILFGER, NIKE, ADIDAS, E OUTRAS MARCAS DIVERSAS',
 "CONFECÇÕES USO ADULTO, LEVI'S, TOMMY HILFGER, GAP, NIKE E OUTRAS MARCAS DIVERSAS",
 'CONFECÇÕES USO ADULTO, THOMMY HILFIGER, CALVIN KLEIN, POLO, ASSN E OUTRAS MARCAS DIVERSAS',
 'CONFECÇÕES USO ADULTO, NIKE, ADDIDAS E OUTRAS MARCAS DIVERSAS',
 'CONFECÇÕES USO ADULTO, aeropostale, levis, gap, puma, calvin klein, tommy, nike, adiddas, lacoste e outras diversas',
 'perfumes - lacoste - ralph lauren - nautica - flight - dolce gabbana, marcas diversas',
 'CONFECÇÕES USO ADULTO, REEBOOK, NIKE, FILA, NEW BALANCE E OUTRAS MARCAS DIVERSAS',
 'CONFECÇÕES USO ADULTO, NIKE, AEROPOSTALE, GAP, POLLO, ADIDAS, E OUTRAS MARCAS DIVERSAS',
 'MOCHILAS - KIPLING, THOMMY HILFGER, SKIP HOP E OUTRAS MARCAS DIVERSAS']

In [15]:
tokenize('Perfume- 125ML - Hugo Boss')

[b'perfume', b'125ml', b'hugo', b'boss']

In [18]:
def monta_sugestoes(planilha):
    lista_dict = []
    planilha.rename(remove_accents)
    planilha = planilha.dropna(subset=['DESCRIÇÃO'])
    for row in planilha.iterrows():
        # print(row[1])
        row_ = {}
        row_['Item'] = row[1]['ITEM']
        row_['Código NCM'] = ''
        row_['Descrição'] = row[1]['DESCRIÇÃO']
        row_['Marca'] = ''
        row_['Modelo'] = ''
        row_['Unid. Medida'] = row[1]['Tipo de UNIDADE']
        row_['País Procedência'] = 249
        row_['País Origem'] = 994
        row_['Quantidade'] = row[1]['Quantidade']
        row_['Valor Unitário'] = ''
        lista_dict.append(row_)
        print(row_['Descrição'])
        doc_scores = bm25n.get_scores(tokenize(row_['Descrição']))
        indices = np.flip(np.argpartition(doc_scores, doc_scores.shape[0] - 3)[-5:])
        new_df = itenstg.iloc[np.flip(indices)]
        for new_row in new_df.iterrows():
            row_ = row_.copy()
            row_['Item'] = '-'
            row_['Descrição'] = new_row[1]['descricao']
            row_['Código NCM'] = new_row[1]['ncm']
            unidade = 'UN' if new_row[1]['unidadedemedida'] == 0 else 'KG'
            row_['Unid. Medida'] = unidade
            row_['Valor Unitário'] = new_row[1]['valor']
            lista_dict.append(row_)
    return pd.DataFrame(lista_dict)

In [24]:
def remove_accents2(input_str):  
    nfkd_form = unicodedata.normalize('NFKD', input_str.lower())
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii.decode('utf-8')

planilha.rename(remove_accents2, axis='columns')

,item,descricao,quantidade,unid. medida
0,1,Pólvora - Hodgon H1000 - 454g,5,UN
1,2,Pólvora - Hodgon CFE 223 - 3630g,2,UN
2,3,Projétil - Nosler - Cal30. 190 g.,250,UN
3,4,Pólvora - Accurate 2015 - 454g,2,UN
4,5,Projétil - Berger - Cal30. 185 g.,200,UN
5,6,"Estojo - Lapua - 7,62 x 51mm",400,UN


In [19]:
diretorio = r'C:\Users\25052288840\Downloads\TGs'
for nome_planilha in os.listdir(diretorio):
    print(diretorio + '\\' + nome_planilha)
    planilha = pd.read_excel(diretorio + '\\' + nome_planilha)
    sugest_df = monta_sugestoes(planilha)
    sugest_df.to_excel(diretorio + '\\TG ' + nome_planilha, index=False)    

C:\Users\25052288840\Downloads\TGs\HASU458015-6 - complemento.xls


TypeError: normalize() argument 2 must be str, not int

In [59]:
bm25n.get_top_n(tokenize('Forno marca: Balmuda, modelo: K01M-KG'), corpus, n=10)

['FORNO ELETRICO, MARCA: MAINSTAYS, MODELO: 1010J-H15',
 'FORNO ELÉTRICO, MARCA: HAMILTON, MODELO: 22703H',
 'FORNO ELÉTRICO BLACK DECKER, MODELO: TO 1313SBD',
 'FORNO ELETRICO 120V/(USADO)',
 'FORNO - AF-250AD2  - KALORIK',
 'FORNO ELETRICO 120V/(USADO)',
 'FORNO MICROONDAS, MARCA: BLACK DECKER, MODELO: EM720CB7',
 'FORNO ELETRICO, MARCA: ELITE GOURMET, MODELO: ETO-4510M',
 'FORNO DE PIZZA ELÉTRICO - GO PLUS',
 'forno elétrico - emeril lagasse']

In [60]:
bm25n.get_top_n(tokenize('Lixadeira Marca: Dexter'), corpus, n=10)

['LIXADEIRA - DCW210 - DE WALT',
 'LIXADEIRA - RK7866 - SHOP SERIES',
 'lixadeira 7800 - porter cable',
 'LIXADEIRA, MARCA DEWALT, MODELO DWL6423',
 'LIXADEIRA DE UNHA - DR-203 - MELODY SUSIE',
 'KIT DE FERRAMENTAS ELÉTRICAS - 01 LIXADEIRA - 01 BATERIA - 01 CARREGADOR - DE WALT',
 'Lixadeira Roto Orbital 5 125mm 20v Brushless DE WALT',
 'LIXADEIRA PROFISSIONAL ORBITAL  DWE6411 AMARELA 50HZ - DEWALT',
 'LIXADEIRA PROFISSIONAL ORBITAL  2.5 AMP ORBITAL SANDER - HYPER TOUGH',
 'LIXADEIRA ELÉTRICA PARA UNHA, BIVOLT, REF DM-13 RF-CB-240 /ORIGEM CHINA']

In [61]:
bm25n.get_top_n(tokenize('Microscópio Digital'), corpus, n=10)

['microscópio 52-piece - erly explorer series - IQ CREW',
 'conversor digital TV  - az',
 'BALANÇA DIGITAL RENPHO',
 'LAREIRA DIGITAL - EF421SP',
 'TERMÔMETRO DIGITAL - DR. TALBOT´S',
 'TERMOMETRO DIGITAL - IHEALTH',
 'BALANÇA DIGITAL - ESCALI',
 'termometro digital - genial',
 'MINI CAMERA DIGITAL - VIVITAR',
 'conversor digital TV ROKU SE  - roku']

In [14]:
bm25n.get_top_n(tokenize('Notebook  Asus'), corpus, n=10)

['NOTEBOOK - ASUS - USADO',
 'NOTEBOOK ASUS VIVOBOOK, MARCA ASUS, MODELO R543M',
 'tablet nexus 7 - C90KBC487109 - ASUS',
 'notebook - HP - usado',
 'NOTEBOOK  (USADO)',
 'NOTEBOOK (USADO)',
 'NOTEBOOK - 15 POLEGADAS 15-BAO15WN - HP',
 'NOTEBOOK  G5CL3BY93 - DELL',
 'NOTEBOOK - DELL - USADO',
 'NOTEBOOK - ADM A8 - LENOVO']

In [15]:
bm25n.get_top_n(tokenize('Lixadeira Dexter'), corpus, n=10)

['LIXADEIRA - DCW210 - DE WALT',
 'LIXADEIRA - RK7866 - SHOP SERIES',
 'lixadeira 7800 - porter cable',
 'LIXADEIRA DE UNHA - DR-203 - MELODY SUSIE',
 'LIXADEIRA PROFISSIONAL ORBITAL  DWE6411 AMARELA 50HZ - DEWALT',
 'Lixadeira Roto Orbital 5 125mm 20v Brushless DE WALT',
 'KIT DE FERRAMENTAS ELÉTRICAS - 01 LIXADEIRA - 01 BATERIA - 01 CARREGADOR - DE WALT',
 'LIXADEIRA PROFISSIONAL ORBITAL  2.5 AMP ORBITAL SANDER - HYPER TOUGH',
 'LIXADEIRA ELÉTRICA PARA UNHA, BIVOLT, REF DM-13 RF-CB-240 /ORIGEM CHINA',
 'LIXADEIRA ELÉTRICA PROFISSIONAL P/UNHA,C/BROCAS, MODELO LG3-1 110V 25000 RPM, REF LJ-CB-147 /JPC']

In [67]:
bm25n.get_top_n(tokenize('Espremedor black modelo: CJ625'), corpus, n=10)

['ESPREMEDOR DE FRUTAS 120/(USADO)',
 'ESPREMEDOR - K37-010 - HOMELEADER',
 'ESPREMEDOR DE FRUTAS 120V/(USADO)',
 'ESPREMEDOR DE FRUTAS, MARCA: CUISINART, MODELO: CCJ500',
 'ESPREMEDOR DE FRUTAS - PKJCR305 - NUTRICHEF',
 'ESPREMEDOR DE FRUTAS - BJE510XL - BREVILLE - USADO',
 'ESPREMEDOR DE FRUTAS - JM480S - JUICEMAN - USADO',
 'APARELHO ESPREMEDOR DE FRUTA ELÉTRICO - ELITE',
 'CAFETEIRA, MARCA: BLACK DECKER, MODELO: CM0750BS',
 'TORRADEIRA , MARCA BLACK + DECKER, MODELO TR1478BD']

In [17]:
SQL = 'SELECT title as ncm, contents as descricao FROM laudo_ncm'
df_ncm = pd.read_sql(SQL, engine)
itensncm = df_ncm.dropna(subset=['ncm'])

In [18]:
print(len(df_ncm), len(itensncm))

10147 10147


In [23]:
corpus_ncm = list(itensncm.descricao.values)
corpus_ncm_normalized = [tokenize(line) for line in corpus_ncm]
bm25n_ncm = BM25Okapi(corpus_ncm_normalized)

In [25]:
bm25n_ncm.get_top_n(tokenize('KIT DE CARTEIRA COM RELÓGIO DE PULSO'), corpus_ncm, n=10)

['--\tNão combinados com um aparelho de gravação ou de reprodução de som, mas combinados com um relógio -\tOutros: Aparelhos receptores para radiodifusão, mesmo combinados num mesmo invólucro, com um aparelho de gravação ou de reprodução de som, ou com um relógio.',
 'Combinado com relógio --\tOutros -\tAparelhos receptores de radiodifusão suscetíveis de funcionarem sem fonte externa de energia: Aparelhos receptores para radiodifusão, mesmo combinados num mesmo invólucro, com um aparelho de gravação ou de reprodução de som, ou com um relógio.',
 'Correias transportadoras ou de transmissão, de matérias têxteis, mesmo impregnadas, revestidas ou recobertas, de plástico, ou estratificadas com plástico ou reforçadas com metal ou com outras matérias.',
 '-\tFechos e armações com fecho, com fechadura Cadeados, fechaduras e ferrolhos (de chave, de segredo ou elétricos), de metais comuns; fechos e armações com fecho, com fechadura, de metais comuns; chaves para estes artigos, de metais comuns.'